In [1]:
################################################################################
#
# Licensed Materials - Property of IBM
# (C) Copyright IBM Corp. 2019
# US Government Users Restricted Rights - Use, duplication disclosure restricted
# by GSA ADP Schedule Contract with IBM Corp.
#
################################################################################


In [2]:
# determine if autoai_libs needs to be updated in the service pod, and if so, install the minimum required version
import subprocess
target_package = 'autoai-libs=='
process = subprocess.run('pip freeze | grep ' + target_package, stdout=subprocess.PIPE, shell=True)
output = process.stdout.decode('ascii')
installed_vrm = ['0','0','0']
if output.startswith(target_package):
    version_string = output[len(target_package):-1]  # remove newline from end too
    installed_vrm = version_string.split('.')
minimum_vrm = ['1','10','3']
update_libs = False
for i, val in enumerate(installed_vrm):
    if int(val) < int(minimum_vrm[i]):
        update_libs = True
        break
if update_libs:
    module_name = '/opt/ibm/.autoAI/autoai_libs-1.10.3-1-cp36-cp36m-linux_x86_64.whl'
    print('attempting pip install of ' + module_name)
    process = subprocess.Popen('pip install ' + module_name, shell=True)
    process.wait()
    print('installation successful')

## IBM AutoAI Auto-Generated Notebook v1.10.5
### Representing Pipeline: P3 from run 1dfb0a2e-6c0e-4c4b-9ee0-9f73a7389ad2

**Note**: Notebook code generated using AutoAI will execute successfully.  If code is modified or reordered, there is no guarantee it will successfully execute, please read our documentation for more information https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/autoai-notebook.html .

Before modifying the pipeline or trying to re-fit the pipeline, consider:
The notebook converts dataframes to numpy arrays before fitting the pipeline (a current restriction of the preprocessor pipeline).
The known_values_list is passed by reference and populated with categorical values during fit of the preprocessing pipeline.  Delete its members before re-fitting.


### 1. Set Up

In [3]:
import sklearn
try:
    import xgboost
except:
    print('xgboost, if needed, will be installed and imported later')
try:
    import lightgbm
except:
    print('lightgbm, if needed, will be installed and imported later')
from sklearn.cluster import FeatureAgglomeration
import numpy
from numpy import nan, dtype, mean
import autoai_libs
from autoai_libs.sklearn.custom_scorers import CustomScorers
import sklearn.ensemble
from autoai_libs.cognito.transforms.transform_utils import TExtras, FC
from autoai_libs.transformers.exportable import *
from autoai_libs.utils.exportable_utils import *
from sklearn.pipeline import Pipeline
known_values_list=[]


lightgbm, if needed, will be installed and imported later


### 2. Compose Pipeline

In [4]:
# The code was removed by Watson Studio for sharing.

re-attempting import of transform_extras from autoai_libs.cognito.transforms
Pipeline successfully instantiated


/opt/conda/envs/Python36/lib/python3.6/site-packages/sklearn/utils/deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)
/opt/conda/envs/Python36/lib/python3.6/site-packages/sklearn/utils/deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


In [5]:
# Metadata used in retrieving data and computing metrics.  Customize as necessary for your environment.
data_source='replace_with_path_and_csv_filename'
target_label_name = _input_metadata['target_label_name']
learning_type = _input_metadata['learning_type']
optimization_metric = _input_metadata['optimization_metric']
random_state = _input_metadata['random_state']
cv_num_folds = _input_metadata['cv_num_folds']
holdout_fraction = _input_metadata['holdout_fraction']
data_provenance = _input_metadata['data_provenance']


In [6]:
import pandas as pd
df = None
if type(data_provenance) is str:
    try:
        df = pd.read_csv(data_provenance) # your data file name here
    except Exception as e:
        print(e)
if df is None:
    try:
        data_location = data_provenance['input_data'][0]
        print('data_location '+ str(data_location))
        import boto3
        session = boto3.session.Session()
        cos = session.client(
            service_name='s3',
            aws_access_key_id=data_location['connection']['access_key_id'],
            aws_secret_access_key=data_location['connection']['secret_access_key'],
            endpoint_url=data_location['connection']['endpoint_url'],
            verify=False
        )
        local_path = data_location['location']['path']
        print('local_path ' + str(local_path))
        cos.download_file(data_location['location']['bucket'],
                     data_location['location']['path'],
                     local_path)
        df = pd.read_csv(local_path) # your data file name here
    except Exception as e:
        print(e)
if df is None:
    raise(ValueError('need location or credential information to read dataframe from COS'))


data_location {'id': '1', 'type': 's3', 'connection': {'access_key_id': '2ed76c071b6448ff9f603f30b3431b8a', 'secret_access_key': '4102030211f936020bf8de5391bfe21e2ca36d23265a6ba3', 'endpoint_url': 'https://s3-api.us-geo.objectstorage.softlayer.net'}, 'location': {'type': 's3', 'bucket': 'predictivemaintenance-donotdelete-pr-mlejlc5nq1ttru', 'path': 'historical.csv'}}
local_path historical.csv


/opt/conda/envs/Python36/lib/python3.6/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/opt/conda/envs/Python36/lib/python3.6/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


In [7]:
target = target_label_name # your target name here
df_y = df[target]


In [8]:
df_X = df.drop(columns=[target])

In [9]:
from sklearn.model_selection import train_test_split

In [10]:
# until the problem type is available in the metadata, use the sklearn type_of_target to determine whether to stratify the holdout split
from sklearn.utils.multiclass import type_of_target
if type_of_target(df_y.values) in ['multiclass', 'binary']:
    X, X_holdout, y, y_holdout = train_test_split(df_X.values, df_y.values, test_size=holdout_fraction, random_state=random_state, stratify=df_y.values)
else:
    X, X_holdout, y, y_holdout = train_test_split(df_X.values, df_y.values, test_size=holdout_fraction, random_state=random_state)


In [11]:
pipeline.fit(X, y)


Pipeline(memory=None,
     steps=[('preprocessor', Pipeline(memory=None,
     steps=[('features', FeatureUnion(n_jobs=None,
       transformer_list=[('categorical', Pipeline(memory=None,
     steps=[('cat_column_selector', NumpyColumnSelector(columns=[0])), ('cat_compress_strings', CompressStrings(activate_flag=True, compress...n'))])), ('estimator', LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=True))])

In [12]:
y_pred = pipeline.predict(X_holdout)


In [13]:
print(y_holdout)
print(y_pred)

[ 8 12  7  4  4  7 13 13  7  4  9  3  8 14  3 10 11  8  4  9  9  8  8 10
 10 10  8 11  9 14 11  4  5  7  6 12  6 12  8  3  6  7  7  5  7  6 10  9
  8  8  8  4 15  9  7  6  9  6 13  3  9  3 10  4  5 10 11  5  5  5 12  5
  7  8  3  4  6  5  5  5  6  9  4  9  6  6  5  7 11 13  9  7  5  6 11  8
  5  6  7 14]
[ 9.726476   9.077933  10.343016   4.7213955  5.569722   5.381709
 10.571469  10.683764   7.489379   6.3740926  9.917762   4.939227
  7.3512125 11.098298   4.507491   9.821881  10.360441   6.741496
  6.2490616  6.823009  10.623696   8.668901   9.632336  11.748581
 10.236459   7.788136   9.386002   8.973754   9.726275  11.255938
  8.890341   4.530704   7.6558933  8.206708   5.060564   8.834135
  7.776186  10.984617   9.570418   4.535057   7.529214   9.044755
  6.064064   4.228711   9.162923   4.915103  11.594592   9.087476
  8.188423   9.624701   7.7277     4.4880695 11.225461   7.680917
  8.312495   5.3793387 11.272545   7.0352373 10.105771   4.457984
  7.6342344  3.909327  10.562271  

In [14]:
from sklearn.metrics import get_scorer
scorer = get_scorer(optimization_metric)
scorer(pipeline, X_holdout, y_holdout)


-1.7428315383058257

In [15]:
#delete CONTENTS of known_values_list before refitting, cloning or cross_validate-ing the pipeline, or previous values will be used.
for i in range(len(known_values_list)):
    del known_values_list[0]

In [16]:
# preprocessor should see all data for cross_validate on the remaining steps to match autoai scores
preprocessor = pipeline.steps[0][1]
preprocessor.fit(X)
X_prep = preprocessor.transform(X)

In [17]:
# remove preprocessor from pipeline
del pipeline.steps[0]

In [18]:
# cross_validate on remaining steps of pipeline
from sklearn.model_selection import cross_validate
cv_results = cross_validate(pipeline, X_prep, y, scoring={optimization_metric:scorer})
import numpy as np
np.mean(cv_results['test_' + optimization_metric])


/opt/conda/envs/Python36/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


-1.7323305093199988

In [19]:
cv_results

/opt/conda/envs/Python36/lib/python3.6/site-packages/sklearn/utils/deprecation.py:125: FutureWarning: You are accessing a training score ('train_neg_root_mean_squared_error'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)


{'fit_time': array([2.08886576, 1.7591331 , 1.73914862]),
 'score_time': array([0.15585518, 0.17067647, 0.20211554]),
 'test_neg_root_mean_squared_error': array([-1.75098564, -1.6831234 , -1.76288249]),
 'train_neg_root_mean_squared_error': array([-1.6643187 , -1.71820565, -1.66904996])}